In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import warnings
warnings.filterwarnings('ignore')
start = time.time()


c:\Users\afias\anaconda3\envs\dev\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# read CSV and create Pandas dataframe
df = pd.read_csv("trip_data.csv")

# preview df
df

,request_datetime,hvfhs_license_num,dispatching_base_num,originating_base_num,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,2023-01-01 01:23:54,HV0003,B03404,B03404,2023-01-01 01:32:20,2023-01-01 01:34:04,2023-01-01 01:45:04,235,18,2.060,...,2.12,0.00,0.0,0.0,14.02,N,N,,N,N
1,2023-01-01 01:58:19,HV0003,B03404,B03404,2023-01-01 02:02:31,2023-01-01 02:02:53,2023-01-01 02:25:52,37,33,4.380,...,3.70,0.00,0.0,0.0,33.24,N,N,,N,N
2,2023-01-01 01:59:09,HV0003,B03404,B03404,2023-01-01 02:02:30,2023-01-01 02:04:02,2023-01-01 02:13:52,232,148,1.390,...,3.02,2.75,0.0,0.0,45.18,N,N,,N,N
3,2023-01-01 02:14:02,HV0003,B03404,B03404,2023-01-01 02:22:59,2023-01-01 02:23:19,2023-01-01 02:31:00,255,256,1.240,...,3.17,0.00,0.0,0.0,31.67,N,N,,N,N
4,2023-01-01 02:16:53,HV0003,B03404,B03404,2023-01-01 02:23:39,2023-01-01 02:24:02,2023-01-01 02:41:11,223,129,2.760,...,2.46,0.00,0.0,0.0,22.54,N,N,,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,2023-12-31 22:08:52,HV0005,B03406,NaN,NaN,2023-12-31 22:20:42,2023-12-31 22:43:17,60,244,4.694,...,5.25,0.00,0.0,0.0,20.30,N,N,N,N,N
11996,2023-12-31 22:17:11,HV0005,B03406,NaN,NaN,2023-12-31 22:23:08,2023-12-31 22:44:52,188,165,4.222,...,3.82,0.00,0.0,0.0,17.96,N,N,N,N,N
11997,2023-12-31 22:30:20,HV0003,B03404,B03404,2023-12-31 22:34:13,2023-12-31 22:36:14,2023-12-31 22:59:56,225,148,5.470,...,2.25,2.75,0.0,0.0,20.55,N,N,N,N,N
11998,2023-12-31 22:31:17,HV0003,B03404,B03404,2023-12-31 22:33:22,2023-12-31 22:35:24,2023-12-31 23:23:07,65,50,6.140,...,3.05,2.75,0.0,0.0,39.12,N,N,N,N,Y


In [7]:
move_column = df.pop("request_datetime")
df.insert(3, "request_datetime", move_column)

In [8]:
df

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2023-01-01 01:23:54,2023-01-01 01:32:20,2023-01-01 01:34:04,2023-01-01 01:45:04,235,18,2.060,...,2.12,0.00,0.0,0.0,14.02,N,N,,N,N
1,HV0003,B03404,B03404,2023-01-01 01:58:19,2023-01-01 02:02:31,2023-01-01 02:02:53,2023-01-01 02:25:52,37,33,4.380,...,3.70,0.00,0.0,0.0,33.24,N,N,,N,N
2,HV0003,B03404,B03404,2023-01-01 01:59:09,2023-01-01 02:02:30,2023-01-01 02:04:02,2023-01-01 02:13:52,232,148,1.390,...,3.02,2.75,0.0,0.0,45.18,N,N,,N,N
3,HV0003,B03404,B03404,2023-01-01 02:14:02,2023-01-01 02:22:59,2023-01-01 02:23:19,2023-01-01 02:31:00,255,256,1.240,...,3.17,0.00,0.0,0.0,31.67,N,N,,N,N
4,HV0003,B03404,B03404,2023-01-01 02:16:53,2023-01-01 02:23:39,2023-01-01 02:24:02,2023-01-01 02:41:11,223,129,2.760,...,2.46,0.00,0.0,0.0,22.54,N,N,,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,HV0005,B03406,NaN,2023-12-31 22:08:52,NaN,2023-12-31 22:20:42,2023-12-31 22:43:17,60,244,4.694,...,5.25,0.00,0.0,0.0,20.30,N,N,N,N,N
11996,HV0005,B03406,NaN,2023-12-31 22:17:11,NaN,2023-12-31 22:23:08,2023-12-31 22:44:52,188,165,4.222,...,3.82,0.00,0.0,0.0,17.96,N,N,N,N,N
11997,HV0003,B03404,B03404,2023-12-31 22:30:20,2023-12-31 22:34:13,2023-12-31 22:36:14,2023-12-31 22:59:56,225,148,5.470,...,2.25,2.75,0.0,0.0,20.55,N,N,N,N,N
11998,HV0003,B03404,B03404,2023-12-31 22:31:17,2023-12-31 22:33:22,2023-12-31 22:35:24,2023-12-31 23:23:07,65,50,6.140,...,3.05,2.75,0.0,0.0,39.12,N,N,N,N,Y


In [9]:
#View total number of datasets
print(len(df))

12000


In [11]:
# access taxi zone csv
location_encoding_csv = pd.read_csv(Path("taxi_zone_lookup.csv"))
# create columns for boroughs and populate with borough names
# create a dictionary of boroughs based on LocationID
borough_id_dict = dict(zip(location_encoding_csv["LocationID"], location_encoding_csv["Borough"]))
# replace location id with borough columns
df["PUborough"] = df["PULocationID"].replace(borough_id_dict)
df["DOborough"] = df["PULocationID"].replace(borough_id_dict)
# convert ID to zone text
# create a dictionary of locations based on LocationID
zone_id_dict = dict(zip(location_encoding_csv["LocationID"], location_encoding_csv["Zone"]))
# replace location ID with zone name
df["PULocationID"].replace(zone_id_dict, inplace=True)
df["DOLocationID"].replace(zone_id_dict, inplace=True)
# preview new columns in df
df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,PUborough,DOborough
0,HV0003,B03404,B03404,2023-01-01 01:23:54,2023-01-01 01:32:20,2023-01-01 01:34:04,2023-01-01 01:45:04,University Heights/Morris Heights,Bedford Park,2.06,...,0.0,0.0,14.02,N,N,,N,N,Bronx,Bronx
1,HV0003,B03404,B03404,2023-01-01 01:58:19,2023-01-01 02:02:31,2023-01-01 02:02:53,2023-01-01 02:25:52,Bushwick South,Brooklyn Heights,4.38,...,0.0,0.0,33.24,N,N,,N,N,Brooklyn,Brooklyn
2,HV0003,B03404,B03404,2023-01-01 01:59:09,2023-01-01 02:02:30,2023-01-01 02:04:02,2023-01-01 02:13:52,Two Bridges/Seward Park,Lower East Side,1.39,...,0.0,0.0,45.18,N,N,,N,N,Manhattan,Manhattan
3,HV0003,B03404,B03404,2023-01-01 02:14:02,2023-01-01 02:22:59,2023-01-01 02:23:19,2023-01-01 02:31:00,Williamsburg (North Side),Williamsburg (South Side),1.24,...,0.0,0.0,31.67,N,N,,N,N,Brooklyn,Brooklyn
4,HV0003,B03404,B03404,2023-01-01 02:16:53,2023-01-01 02:23:39,2023-01-01 02:24:02,2023-01-01 02:41:11,Steinway,Jackson Heights,2.76,...,0.0,0.0,22.54,N,N,,N,N,Queens,Queens


In [12]:
# df info
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   hvfhs_license_num     12000 non-null  object 
 1   dispatching_base_num  12000 non-null  object 
 2   originating_base_num  8603 non-null   object 
 3   request_datetime      12000 non-null  object 
 4   on_scene_datetime     8603 non-null   object 
 5   pickup_datetime       12000 non-null  object 
 6   dropoff_datetime      12000 non-null  object 
 7   PULocationID          12000 non-null  object 
 8   DOLocationID          12000 non-null  object 
 9   trip_miles            12000 non-null  float64
 10  trip_time             12000 non-null  int64  
 11  base_passenger_fare   12000 non-null  float64
 12  tolls                 12000 non-null  float64
 13  bcf                   12000 non-null  float64
 14  sales_tax             12000 non-null  float64
 15  congestion_surcharg

In [13]:
# Renaming columns
column_dict = {
    'hvfhs_license_num': 'company',
    'dispatching_base_num': 'd1',
    'originating_base_num': 'd2',
    'request_datetime': 'request_time',
    'on_scene_datetime': 'on_scene_time',
    'pickup_datetime': 'pickup_time',
    'dropoff_datetime': 'dropoff_time',
    'PULocationID': 'pickup_location',
    'DOLocationID': 'dropoff_location',
    'trip_miles': 'trip_length',
    'trip_time': 'trip_time_seconds',
    'base_passenger_fare': 'base_passenger_fare',
    'tolls': 'tolls',
    'bcf': 'bcf',
    'sales_tax': 'sales_tax',
    'congestion_surcharge': 'congestion_surcharge',
    'airport_fee': 'airport_fee',
    'tips': 'tips',
    'driver_pay': 'driver_pay',
    'shared_request_flag': 'd3',
    'shared_match_flag': 'd4',
    'access_a_ride_flag': 'd5',
    'wav_request_flag': 'd6',
    'wav_match_flag': 'd7',
    }

In [14]:
#Drop and rename columns
columns_dropout = ['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7']

df = df.rename(columns=column_dict)


df = df.drop(columns=columns_dropout)

In [15]:
# Remapping of values & Custom Measures
#business mapping
replacements = {
    'HV0002': 'Juno',
    'HV0003': 'Uber',
    'HV0004': 'Via',
    'HV0005': 'Lyft'
}

#replace values in business column
df['company'] = df['company'].map(replacements).fillna(df['company'])

In [16]:
# Check for null datasets
df.isnull().sum()

company                    0
request_time               0
on_scene_time           3397
pickup_time                0
dropoff_time               0
pickup_location            0
dropoff_location           0
trip_length                0
trip_time_seconds          0
base_passenger_fare        0
tolls                      0
bcf                        0
sales_tax                  0
congestion_surcharge       0
airport_fee                0
tips                       0
driver_pay                 0
PUborough                  0
DOborough                  0
dtype: int64

In [17]:
#View null values
df[df[['on_scene_time']].isnull()]

,company,request_time,on_scene_time,pickup_time,dropoff_time,pickup_location,dropoff_location,trip_length,trip_time_seconds,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,PUborough,DOborough
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# drop all null values
df = df.dropna()

In [19]:
display(df.head())
display(df.tail())

,company,request_time,on_scene_time,pickup_time,dropoff_time,pickup_location,dropoff_location,trip_length,trip_time_seconds,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,PUborough,DOborough
0,Uber,2023-01-01 01:23:54,2023-01-01 01:32:20,2023-01-01 01:34:04,2023-01-01 01:45:04,University Heights/Morris Heights,Bedford Park,2.06,660,23.87,0.0,0.72,2.12,0.00,0.0,0.0,14.02,Bronx,Bronx
1,Uber,2023-01-01 01:58:19,2023-01-01 02:02:31,2023-01-01 02:02:53,2023-01-01 02:25:52,Bushwick South,Brooklyn Heights,4.38,1379,41.66,0.0,1.25,3.70,0.00,0.0,0.0,33.24,Brooklyn,Brooklyn
2,Uber,2023-01-01 01:59:09,2023-01-01 02:02:30,2023-01-01 02:04:02,2023-01-01 02:13:52,Two Bridges/Seward Park,Lower East Side,1.39,590,34.03,0.0,1.02,3.02,2.75,0.0,0.0,45.18,Manhattan,Manhattan
3,Uber,2023-01-01 02:14:02,2023-01-01 02:22:59,2023-01-01 02:23:19,2023-01-01 02:31:00,Williamsburg (North Side),Williamsburg (South Side),1.24,461,35.71,0.0,1.07,3.17,0.00,0.0,0.0,31.67,Brooklyn,Brooklyn
4,Uber,2023-01-01 02:16:53,2023-01-01 02:23:39,2023-01-01 02:24:02,2023-01-01 02:41:11,Steinway,Jackson Heights,2.76,1029,27.69,0.0,0.83,2.46,0.00,0.0,0.0,22.54,Queens,Queens


,company,request_time,on_scene_time,pickup_time,dropoff_time,pickup_location,dropoff_location,trip_length,trip_time_seconds,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,PUborough,DOborough
11992,Uber,2023-12-31 20:59:00,2023-12-31 21:05:06,2023-12-31 21:05:17,2023-12-31 21:09:18,Borough Park,Sunset Park East,0.66,241,8.87,0.0,0.24,0.79,0.00,0.0,0.0,5.40,Brooklyn,Brooklyn
11993,Uber,2023-12-31 21:43:01,2023-12-31 21:52:51,2023-12-31 21:54:05,2023-12-31 22:00:24,West Concourse,Highbridge,0.88,379,12.15,0.0,0.33,1.08,0.00,0.0,0.0,7.61,Bronx,Bronx
11994,Uber,2023-12-31 21:45:55,2023-12-31 21:48:42,2023-12-31 21:49:21,2023-12-31 21:59:29,Two Bridges/Seward Park,Financial District North,3.26,608,15.40,0.0,0.42,1.37,2.75,0.0,3.0,10.48,Manhattan,Manhattan
11997,Uber,2023-12-31 22:30:20,2023-12-31 22:34:13,2023-12-31 22:36:14,2023-12-31 22:59:56,Stuyvesant Heights,Lower East Side,5.47,1422,25.40,0.0,0.70,2.25,2.75,0.0,0.0,20.55,Brooklyn,Brooklyn
11998,Uber,2023-12-31 22:31:17,2023-12-31 22:33:22,2023-12-31 22:35:24,2023-12-31 23:23:07,Downtown Brooklyn/MetroTech,Clinton West,6.14,2863,34.31,0.0,0.94,3.05,2.75,0.0,0.0,39.12,Brooklyn,Brooklyn


In [20]:
df.dtypes

company                  object
request_time             object
on_scene_time            object
pickup_time              object
dropoff_time             object
pickup_location          object
dropoff_location         object
trip_length             float64
trip_time_seconds         int64
base_passenger_fare     float64
tolls                   float64
bcf                     float64
sales_tax               float64
congestion_surcharge    float64
airport_fee             float64
tips                    float64
driver_pay              float64
PUborough                object
DOborough                object
dtype: object

In [21]:
df.head()

,company,request_time,on_scene_time,pickup_time,dropoff_time,pickup_location,dropoff_location,trip_length,trip_time_seconds,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,PUborough,DOborough
0,Uber,2023-01-01 01:23:54,2023-01-01 01:32:20,2023-01-01 01:34:04,2023-01-01 01:45:04,University Heights/Morris Heights,Bedford Park,2.06,660,23.87,0.0,0.72,2.12,0.00,0.0,0.0,14.02,Bronx,Bronx
1,Uber,2023-01-01 01:58:19,2023-01-01 02:02:31,2023-01-01 02:02:53,2023-01-01 02:25:52,Bushwick South,Brooklyn Heights,4.38,1379,41.66,0.0,1.25,3.70,0.00,0.0,0.0,33.24,Brooklyn,Brooklyn
2,Uber,2023-01-01 01:59:09,2023-01-01 02:02:30,2023-01-01 02:04:02,2023-01-01 02:13:52,Two Bridges/Seward Park,Lower East Side,1.39,590,34.03,0.0,1.02,3.02,2.75,0.0,0.0,45.18,Manhattan,Manhattan
3,Uber,2023-01-01 02:14:02,2023-01-01 02:22:59,2023-01-01 02:23:19,2023-01-01 02:31:00,Williamsburg (North Side),Williamsburg (South Side),1.24,461,35.71,0.0,1.07,3.17,0.00,0.0,0.0,31.67,Brooklyn,Brooklyn
4,Uber,2023-01-01 02:16:53,2023-01-01 02:23:39,2023-01-01 02:24:02,2023-01-01 02:41:11,Steinway,Jackson Heights,2.76,1029,27.69,0.0,0.83,2.46,0.00,0.0,0.0,22.54,Queens,Queens


## Encode the dataset categorical variables using OneHotEncoder, and then place the encoded variables into a new DataFrame.

In [ ]:
# Create a list of categorical variables
categorical_variables = list(df.dtypes[df.dtypes=='object'].index)

In [ ]:
# Display the categorical variables list
categorical_variables

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

In [ ]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(df[categorical_variables])

In [ ]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns=enc.get_feature_names_out(categorical_variables)
)

# Review the DataFrame
encoded_df.head()

## Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

In [ ]:
# Encode the non categorical variables using OneHotEncoder
non_categorical_variables =list(df.dtypes[df.dtypes!='object'].index)
non_categorical_variables

In [ ]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat([df[non_categorical_variables], encoded_df], axis = 1)

# Review the DataFrame
encoded_df.head()

In [ ]:
encoded_df.tail()

In [ ]:
encoded_df.dropna()